<a href="https://colab.research.google.com/github/moustafa-7/League-of-legends_Bigdata/blob/main/Bigdata_miniprojects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install p7zip-full

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [ ]:
!7z e drive/MyDrive/matches.jsonl.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 554569618 bytes (529 MiB)

Extracting archive: drive/MyDrive/matches.jsonl.7z
--
Path = drive/MyDrive/matches.jsonl.7z
Type = 7z
Physical Size = 554569618
Headers Size = 130
Method = LZMA2:24
Solid = -
Blocks = 1

  0%      0% - matches.jsonl                      1% - matches.jsonl                      2% - matches.jsonl                      3% - matches.jsonl                      4% - matches.jsonl                      5% - matches.jsonl                      6% - matches.

In [ ]:
!head -3000 matches.jsonl > drive/MyDrive/small_matches.jsonl

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
import pyspark
import random

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LOL").getOrCreate()
sc = spark.sparkContext
d = spark.read.json("drive/MyDrive/small_matches.json", multiLine=True)
# d = spark.read.json("drive/MyDrive/small_matches.jsonl", multiLine=False)

In [ ]:
no_matches = d.count()
print(no_matches)

1339


## 1. Champions win - pick - ban rates

In [ ]:
def champions(row):
  c = []
  bans = []
  for p in row["participants"]:
    if(p["teamId"] == "Blue"): team_number = 0
    else: team_number = 1
    c.append((p["championId"], row["teams"][team_number]["win"]))
  
  for t in row["teams"]:
    for b in t["bans"]:
      if(b["championId"] != None):
        c.append((b["championId"],"ban"))
      # else:
      #   c.append((b["championid"],"ban"))

  return c

In [ ]:
champions = d.rdd.flatMap(champions)

In [ ]:
import time
def lookup(champions):
  for c in champions.collect():
    print(c)
    time.sleep(0.1)

In [ ]:
by_champion = champions.groupByKey().mapValues(list)

In [ ]:
# for c, val in by_champion.collect():
#     print(c,val)
#     time.sleep(0.1)

In [ ]:
def compute_champion_rates(val):
    ban_count = val.count("ban")
    win_count = val.count("Win")
    lose_count = val.count("Fail")
    total_played_games = win_count+lose_count
    if(total_played_games):
      return [ban_count/no_matches, win_count/total_played_games, lose_count/total_played_games]
    else:
      return [ban_count/no_matches, "never_played", "never_played"]

In [ ]:
win_pick_ban_rates = by_champion.map(lambda x:(x[0], compute_champion_rates(x[1])))

In [ ]:
for c in win_pick_ban_rates.collect()[:10]:
    print(c)
    time.sleep(0.1)

('Karma', [0.006333333333333333, 0.4230769230769231, 0.5769230769230769])
('Ezreal', [0.09966666666666667, 0.5312208760484622, 0.46877912395153776])
('Nunu & Willump', [0.007, 0.5, 0.5])
('Mordekaiser', [0.008666666666666666, 0.35, 0.65])
('Aurelion Sol', [0.006666666666666667, 0.4942528735632184, 0.5057471264367817])
('Heimerdinger', [0.004666666666666667, 0.49238578680203043, 0.5076142131979695])
('Thresh', [0.2663333333333333, 0.4961439588688946, 0.5038560411311054])
('Ekko', [0.10533333333333333, 0.4911111111111111, 0.5088888888888888])
('Rengar', [0.2833333333333333, 0.48638132295719844, 0.5136186770428015])
('Yasuo', [0.38433333333333336, 0.4731182795698925, 0.5268817204301075])


## 2. Champion Synergies or Duos

In [ ]:
import itertools
def champion_duos(row):
  duos = []
  team1 = []
  team2 = []
  is_team1_win = False
  is_team2_win = False
  for p in row["participants"]:
    if(p["teamId"] == "Blue"): 
      team_number = 0
    else: 
      team_number = 1
    
    if(team_number == 0):
      team1.append(p["championId"])
      is_team1_win = p["stats"]["win"]
    else:
      team2.append(p["championId"])
      is_team2_win = p["stats"]["win"]
  
  team1 = sorted(team1)
  team2 = sorted(team2)
  for c in list(itertools.combinations(team1, 2)):
    if is_team1_win:
      duos.append((c, 1.5))
    else:
      duos.append((c, 1))

  for c in list(itertools.combinations(team2, 2)):
    if is_team2_win:
      duos.append((c, 1.5))
    else:
      duos.append((c, 1))

  
  return duos

In [ ]:
duos = d.rdd.flatMap(champion_duos)

In [ ]:
from operator import add
duos_count = duos.reduceByKey(add)

In [ ]:
# for c in duos_count.collect():
#     print(c)
#     time.sleep(0.01)

top_duos = duos_count.top(50, key=lambda x: x[1])
for c in top_duos[:10]:
    print(c)
    time.sleep(0.01)

(('Ezreal', 'Yuumi'), 99.0)
(('Aphelios', 'Thresh'), 87.5)
(('Ezreal', 'Sett'), 73.0)
(('Thresh', 'Varus'), 70.5)
(('Graves', 'Thresh'), 70.0)
(('Ezreal', 'Graves'), 67.0)
(('Ezreal', 'Lee Sin'), 65.5)
(('Lee Sin', 'Thresh'), 64.0)
(('Aphelios', 'Lulu'), 61.5)
(('Lee Sin', 'Syndra'), 60.0)


## 3. Item Win - pick rates

In [ ]:
def get_items(row):
  c = []
  bans = []
  for p in row["participants"]:
    for i in range(7):
      c.append((p["stats"]["item"+str(i)]["name"], p["stats"]["win"]))
  return c

In [ ]:
items = d.rdd.flatMap(get_items).groupByKey().mapValues(list)

In [ ]:

print(len(items.collect()))

190


In [ ]:
def compute_item_rates(val):
  win_count = val.count(True)
  lose_count = val.count(False)
  total_played_games = win_count+lose_count
  return [win_count/total_played_games, len(val)/no_matches]

In [ ]:
item_rates = items.map(lambda x:(x[0], compute_item_rates(x[1])))

In [ ]:
print("item       win_rate        pick_rate")
for item in item_rates.collect()[:10]:
    print(item)
    time.sleep(0.01)

item       win_rate        pick_rate
('Shard of True Ice', [0.5265423242467718, 0.520537714712472])
("Athene's Unholy Grail", [0.5107033639143731, 0.48842419716206126])
('Ardent Censer', [0.5220458553791887, 0.4234503360716953])
("Mercury's Treads", [0.53621103117506, 1.5571321882001494])
('Cloth Armor', [0.40292275574112735, 0.3577296489917849])
('Sapphire Crystal', [0.15384615384615385, 0.02912621359223301])
('Oracle Lens', [0.5067150635208711, 4.1150112023898435])
('Bilgewater Cutlass', [0.47368421052631576, 0.1135175504107543])
('Iceborn Gauntlet', [0.5581395348837209, 0.22479462285287527])
('Muramana', [0.5550314465408805, 0.4749813293502614])


## 4. Item synergies (with champion, with class)

In [ ]:
import json

classes = open("drive/MyDrive/champion_class.json")
champion_classes = json.load(classes)

In [ ]:
def tags(champion_id):
  for c in champion_classes:
    if champion_id==c["championId"]:
      return c["tags"]

In [ ]:
print(*tags("Ahri"))

Mage Assassin


In [ ]:
def get_items_champion(row):
  c = []
  for p in row["participants"]:
    for i in range(7):
      # c.append((p["stats"]["item"+str(i)]["name"], p["stats"]["win"], p["championId"], *tags(p["championId"])))
      if(tags(p["championId"])):
        c.append((p["stats"]["item"+str(i)]["name"], (p["stats"]["win"], p["championId"],*tags(p["championId"]))))
  return c

In [ ]:
items_champion = d.rdd.flatMap(get_items_champion).groupByKey().mapValues(list)

In [ ]:
for item in items_champion.collect()[:10]:
    print(item)
    time.sleep(0.1)

# print(len(items_champion.collect()))

('Shard of True Ice', [(False, 'Karma', 'Mage', 'Support'), (True, 'Karma', 'Mage', 'Support'), (False, 'Soraka', 'Support', 'Mage'), (True, 'Morgana', 'Mage', 'Support'), (True, 'Karma', 'Mage', 'Support'), (False, 'Zyra', 'Mage', 'Support'), (False, 'Karma', 'Mage', 'Support'), (True, 'Yuumi', 'Support', 'Mage'), (False, 'Gragas', 'Fighter', 'Mage'), (True, 'Janna', 'Support', 'Mage'), (True, 'Bard', 'Support', 'Mage'), (True, 'Swain', 'Mage', 'Fighter'), (True, 'Bard', 'Support', 'Mage'), (True, 'Rakan', 'Support'), (True, 'Bard', 'Support', 'Mage'), (False, 'Janna', 'Support', 'Mage'), (True, 'Soraka', 'Support', 'Mage'), (False, 'Lulu', 'Support', 'Mage'), (False, 'Yuumi', 'Support', 'Mage'), (False, 'Morgana', 'Mage', 'Support'), (False, 'Morgana', 'Mage', 'Support'), (False, 'Yuumi', 'Support', 'Mage'), (False, 'Yuumi', 'Support', 'Mage'), (False, 'Soraka', 'Support', 'Mage'), (False, 'Janna', 'Support', 'Mage'), (False, 'Yuumi', 'Support', 'Mage'), (True, 'Lulu', 'Support', 'Ma

In [ ]:
def champion_item_synergies(row):
  champion_dict= {}
  for val in row[1]:
    if(val[1] in champion_dict):
      champion_dict[val[1]] += 1
    else:
      champion_dict[val[1]] = 1
    
    if(val[0] == True):
            champion_dict [val[1]] += 0.5

  c = []
  for k, r in sorted(champion_dict.items(), key=lambda x:x[1], reverse=True):
    c.append((k, r))

  return  c[:10]

In [ ]:
champion_item_synergy = items_champion.flatMap(lambda x:(x[0] ,champion_item_synergies(x)))

In [ ]:
for item in champion_item_synergy.collect()[:10]:
    print(item)
    time.sleep(0.1)

Shard of True Ice
('Shard of True Ice', [('Yuumi', 149.0), ('Lulu', 127.0), ('Karma', 123.0), ('Bard', 104.0), ('Janna', 92.5), ('Morgana', 58.0), ('Soraka', 33.0), ('Nami', 27.5), ('Fiddlesticks', 26.5), ('Brand', 21.0)])
Athene's Unholy Grail
("Athene's Unholy Grail", [('Yuumi', 224.5), ('Karma', 173.5), ('Janna', 107.0), ('Lulu', 90.5), ('Soraka', 45.5), ('Bard', 39.5), ('Ivern', 28.0), ('Rakan', 27.0), ('Nami', 25.5), ('Nidalee', 19.5)])
Ardent Censer
('Ardent Censer', [('Lulu', 278.0), ('Yuumi', 110.5), ('Karma', 103.5), ('Janna', 68.0), ('Soraka', 27.0), ('Ivern', 26.5), ('Rakan', 26.5), ('Nami', 24.5), ('Morgana', 19.5), ('Sona', 9.5)])
Mercury's Treads
("Mercury's Treads", [('Graves', 122.5), ('Sylas', 106.0), ('Irelia', 87.0), ('Renekton', 82.5), ('Sett', 70.5), ('Riven', 69.5), ('Bard', 67.5), ('Jayce', 60.0), ('Ekko', 57.5), ('Jax', 52.5)])
Cloth Armor
('Cloth Armor', [('Thresh', 49.0), ('Lulu', 32.5), ('Blitzcrank', 24.0), ('Sett', 21.5), ('Malphite', 18.5), ('Trundle', 18.

In [ ]:
def class_item_synergies(row):
  class_dict= {}
  for val in row[1]:
    for cls in val[2:]:
      if(cls in class_dict):
        class_dict[cls] += 1
      else:
        class_dict[cls] = 1
    
      if(val[0] == True):
        class_dict[cls] += 0.5

  c = []
  for k, r in sorted(class_dict.items(), key=lambda x:x[1], reverse=True):
    c.append((k, r))

  return (row[0], c[:3])

In [ ]:
class_item_synergy = items_champion.flatMap(lambda x: class_item_synergies(x))

In [ ]:
for item in class_item_synergy.collect()[:10]:
    print(item)
    time.sleep(0.1)

Shard of True Ice
[('Mage', 837.5), ('Support', 818.0), ('Fighter', 10.0)]
Athene's Unholy Grail
[('Support', 801.5), ('Mage', 785.5), ('Assassin', 19.5)]
Ardent Censer
[('Support', 715.0), ('Mage', 682.0), ('Fighter', 6.5)]
Mercury's Treads
[('Fighter', 1271.5), ('Tank', 760.5), ('Assassin', 710.0)]
Cloth Armor
[('Fighter', 320.0), ('Tank', 265.5), ('Support', 169.5)]


## 5. Item suggestion

In [ ]:
def champion_item(row):
  c = []
  for p in row["participants"]:
    # c.append(p["championId"])
    for i in range(7):
      if not p["stats"]["item"+str(i)]["into"] and tags(p["championId"]):
        c.append((p["championId"], ( p["stats"]["item"+str(i)]["name"], p["stats"]["win"], *tags(p["championId"]) )))
  return  c

In [ ]:
c_item_class = d.rdd.flatMap(champion_item).groupByKey().mapValues(list)

In [ ]:
for item in c_item_class.collect():
    print(item)
    time.sleep(0.1)

('Karma', [('Shard of True Ice', False, 'Mage', 'Support'), ("Athene's Unholy Grail", False, 'Mage', 'Support'), ('Ardent Censer', False, 'Mage', 'Support'), ("Mercury's Treads", False, 'Mage', 'Support'), ('Oracle Lens', False, 'Mage', 'Support'), ("Sorcerer's Shoes", True, 'Mage', 'Support'), ('Control Ward', True, 'Mage', 'Support'), ('Shard of True Ice', True, 'Mage', 'Support'), ("Athene's Unholy Grail", True, 'Mage', 'Support'), ('Ardent Censer', True, 'Mage', 'Support'), ('Oracle Lens', True, 'Mage', 'Support'), ('Shard of True Ice', True, 'Mage', 'Support'), ('Ardent Censer', True, 'Mage', 'Support'), (None, True, 'Mage', 'Support'), ('Boots of Mobility', True, 'Mage', 'Support'), (None, True, 'Mage', 'Support'), ("Athene's Unholy Grail", True, 'Mage', 'Support'), ('Oracle Lens', True, 'Mage', 'Support'), ('Frostfang', False, 'Mage', 'Support'), ('Control Ward', False, 'Mage', 'Support'), ('Total Biscuit of Everlasting Will', False, 'Mage', 'Support'), ('Boots of Mobility', Fal

KeyboardInterrupt: ignored

In [ ]:
def champion_item_suggest(row):
  item_dict= {}
  for val in row[1]:
    if(val[0] in item_dict):
      item_dict[val[0]] += 1
    else:
      item_dict[val[0]] = 1
    
    if(val[0] == True):
            item_dict [val[0]] += 1

  c = []
  for k, r in sorted(item_dict.items(), key=lambda x:x[1], reverse=True):
    c.append((k, r))

  
  return (row[0], c[:10])

In [ ]:
champion_item_suggestion = c_item_class.flatMap(lambda x: champion_item_suggest(x))

In [ ]:
for item in champion_item_suggestion.collect():
    print(item)
    time.sleep(0.1)

Karma
[('Oracle Lens', 172), ("Athene's Unholy Grail", 146), (None, 125), ('Shard of True Ice', 104), ('Ionian Boots of Lucidity', 89), ('Ardent Censer', 86), ('Control Ward', 83), ('Bulwark of the Mountain', 33), ('Boots of Swiftness', 30), ("Sorcerer's Shoes", 26)]
Ezreal
[('Farsight Alteration', 416), ('Muramana', 412), ('Ionian Boots of Lucidity', 357), (None, 281), ("Doran's Blade", 250), ('Warding Totem (Trinket)', 68), ('Manamune', 54), ('Control Ward', 51), ("Doran's Shield", 46), ('Ninja Tabi', 44)]
Mordekaiser
[('Warding Totem (Trinket)', 47), (None, 42), ("Liandry's Torment", 29), ('Hextech Protobelt-01', 27), ('Ninja Tabi', 25), ("Doran's Shield", 22), ("Mercury's Treads", 16), ("Nashor's Tooth", 16), ("Rylai's Crystal Scepter", 10), ('Farsight Alteration', 4)]
Heimerdinger
[("Sorcerer's Shoes", 71), (None, 63), ('Farsight Alteration', 56), ("Doran's Ring", 54), ('Warding Totem (Trinket)', 21), ('Morellonomicon', 17), ('Control Ward', 11), ("Liandry's Torment", 8), ("Banshe